In [1]:
import pandas as pd

vineyards = pd.read_csv('vineyard.csv')
vineyards.head()

,Unnamed: 0,vineyard_id,name,appelation,country,region
0,0,0,Nicosia,Vulkà Bianco,Italy,Etna
1,1,1,St. Julian,Reserve Late Harvest,US,Lake Michigan Shore
2,2,2,Sweet Cheeks,Vintner's Reserve Wild Child Block,US,Willamette Valley
3,3,3,Tandem,Ars In Vitro,Spain,Navarra
4,4,4,Terre di Giurfo,Belsito,Italy,Vittoria


In [60]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import json

def scrape_cellar_tracker_search(urls):
    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(options=chrome_options)

    wine_list = []

    for url, appelation in urls:
        
        driver.get(url)

        time.sleep(10)
        
        # Get page source after JS rendering
        html = driver.page_source
        
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find("table", {"id": "main_table"})
        
        # Parse each table row
        try:
            for row in table.find_all("tr")[1:]:
                cols = row.find_all("td")
                if len(cols) < 2:
                    continue
            
                wine_type = cols[0].get_text(strip=True)
            
                name_block = cols[1]
                name = name_block.find("h3").get_text(strip=True)
                loc = name_block.find("span", class_="el loc").get_text(strip=True)
            
                qty = cols[2].find("span", class_="el gty")
                quantity = qty.get_text(strip=True) if qty else ""
            
                score_span = cols[3].find("span", class_="el scr")
                score = score_span.get_text(strip=True) if score_span else ""

                scraped_wine = {
                    "type": wine_type,
                    "name": name,
                    "location": loc,
                    "appelation":appelation,
                    "quantity": quantity,
                    "score": score
                }
                wine_list.append(scraped_wine)
                with open("wines.json", 'r') as f:
                    existing_data = json.load(f)
                existing_data.append(scraped_wine)
                with open("wines.json", "w") as json_file:
                    json.dump(existing_data, json_file)
        except AttributeError:
            continue
    driver.quit()
    return wine_list

In [49]:
searches = []
for index, row in vineyards.iterrows():
    toSearch = "+".join([row["name"], row["region"], row["appelation"], row["country"]])
    searches.append(["https://www.cellartracker.com/list.asp?fInStock=0&iUserOverride=0&Table=List&szSearch=" + toSearch, row["appelation"]])

In [66]:
wines = scrape_cellar_tracker_search(searches[200:300])

In [83]:
with open("wines.json", "r") as f:
    wines = json.load(f)

In [84]:
wines = pd.DataFrame(wines)
wines.rename(columns={"type":"classification", "score":"rating"}, inplace=True)

In [90]:
wines["year"] = wines["name"].str[:4]

In [93]:
wines.drop("quantity", inplace=True, axis=1)

In [99]:
wines['rating_out_of_5'] = (
    wines['rating']
    .str.extract(r'(\d+)')        # Extract digits
    .astype(float)                # Convert to float (NaN if no digits)
    .div(20)                      # Scale to 5-star rating
)


In [103]:
wines.drop("rating", inplace=True, axis=1)

In [107]:
wines.rename(columns={"rating_out_of_5":"rating"}, inplace=True)

In [108]:
wines

,classification,name,location,appelation,year,rating
0,White,2014 Cantine Nicosia Etna Vulka Bianco,Etna DOC,Vulkà Bianco,2014,4.50
1,White,2023 Cantine Nicosia Etna Vulka Bianco,Etna DOC,Vulkà Bianco,2023,NaN
2,White,2022 Cantine Nicosia Etna Vulka Bianco,Etna DOC,Vulkà Bianco,2022,4.50
3,White,2021 Cantine Nicosia Etna Vulka Bianco,Etna DOC,Vulkà Bianco,2021,4.30
4,White,2020 Cantine Nicosia Etna Vulka Bianco,Etna DOC,Vulkà Bianco,2020,NaN
...,...,...,...,...,...,...
1777,White,2018 Raptor Ridge Grüner Veltliner Tuscowallam...,Chehalem Mountains,Estate,2018,4.25
1778,White,2017 Raptor Ridge Grüner Veltliner Tuscowallam...,Chehalem Mountains,Estate,2017,NaN
1779,White,2016 Raptor Ridge Grüner Veltliner Tuscowallam...,Chehalem Mountains,Estate,2016,NaN
1780,Red,2021 Raptor Ridge Pinot Noir Estate,Chehalem Mountains,Estate,2021,NaN


In [109]:
wines.to_csv("wines.csv")

In [ ]:
wines["name_short"] = wines["name"].str[5:]

wines.drop("name", inplace=True, axis=1)

In [13]:
wines.drop("Unnamed: 0", axis=1, inplace=True)

In [15]:
wines.rename(columns={"name_short":"name"}, inplace=True)

In [18]:
wines.to_csv("wines.csv")

In [1]:
import pandas as pd

wines = pd.read_csv("wine.csv")

In [10]:
from openai import OpenAI
client = OpenAI(api_key="")

food_pairings = []
for wine_id, name in wines[["wine_id", "name"]].unique().itertuples(index=False):
    response = client.responses.create(
        model="gpt-4o-mini",
        input=f'Give me the name of the best food pairing that goes with this wine:{name}'
    )
    print(response.output_text)
    
    food_pairings.append({"wine_id":wine_id, "name":result})
    break

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}